In [1]:
!pip install -q "transformers==4.44.2" sentencepiece accelerate nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 106.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 92.0 MB/s eta 0:00:00


In [2]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
!pip install -q "summac==0.0.4" --no-deps


In [4]:
import types, sys

# 给 transformers.models.deprecated.ernie_m 做一个假的空模块
dummy_module = types.ModuleType("transformers.models.deprecated.ernie_m")
sys.modules["transformers.models.deprecated.ernie_m"] = dummy_module

In [5]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForSeq2SeqLM
import torch, numpy as np

print("numpy:", np.__version__)

MODEL_ID = "mercuryujia/bart-large-multi-news"

# 1. 修改 config，让 early_stopping 不为 None
config = AutoConfig.from_pretrained(MODEL_ID)
config.early_stopping = True

# 2. 慢版 tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=False)

# 3. 使用修改过的 config 加载模型
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID, config=config)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


numpy: 2.0.2


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:606: UserWarning: `num_beams` is set to None - defaulting to 1.
  warnings.warn("`num_beams` is set to None - defaulting to 1.", UserWarning)
/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


generation_config.json:   0%|          | 0.00/368 [00:00<?, ?B/s]

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [6]:
from transformers import GenerationConfig

source_document = """
President Joe Biden met with congressional leaders at the White House on Tuesday
to discuss a new economic relief package. The meeting comes as inflation remains
high and lawmakers are divided over the size of government spending.

Meanwhile, the Federal Reserve signaled it may slow down the pace of interest
rate hikes later this year if price increases continue to cool.

Economists say the negotiations in Washington could have major consequences
for financial markets and for American households.
"""

K = 4

gen_config = GenerationConfig(
    bos_token_id=0,
    decoder_start_token_id=2,
    early_stopping=True,
    eos_token_id=[2],
    forced_bos_token_id=0,
    forced_eos_token_id=2,
    length_penalty=2.0,
    max_length=142,
    min_length=56,
    no_repeat_ngram_size=3,
    num_beams=4,
    pad_token_id=1,
)

inputs = tokenizer(
    source_document,
    max_length=1024,
    truncation=True,
    return_tensors="pt"
).to(device)

with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        generation_config=gen_config,
        num_return_sequences=K,
    )

candidate_summaries = tokenizer.batch_decode(output_ids, skip_special_tokens=True)

print("=== 生成的候选摘要 ===")
for i, s in enumerate(candidate_summaries):
    print(f"[Cand {i}] {s}\n")

=== 生成的候选摘要 ===
[Cand 0] – Joe Biden met with congressional leaders at the White House today to discuss a new economic relief package, reports the AP. The meeting comes as inflation remains high and lawmakers are divided over the size of government spending. Meanwhile, the Federal Reserve signaled it may slow down the pace of interest rate hikes later this year if price increases continue to cool.

[Cand 1] – Joe Biden met with congressional leaders at the White House today to discuss a new economic relief package, reports the Hill. The meeting comes as inflation remains high and lawmakers are divided over the size of government spending. Meanwhile, the Federal Reserve signaled it may slow down the pace of interest rate hikes later this year if price increases continue to cool.

[Cand 2] – Joe Biden met with congressional leaders at the White House today to discuss a new economic relief package, reports the AP. The meeting comes as inflation remains high, and lawmakers are divided over

In [7]:
import types, sys
from transformers import PretrainedConfig

# Create a dummy module for transformers.models.deprecated.ernie_m
dummy_module = types.ModuleType("transformers.models.deprecated.ernie_m")

class ErnieMConfig(PretrainedConfig):
    model_type = "ernie_m"

# Expose ErnieMConfig in that dummy module
setattr(dummy_module, "ErnieMConfig", ErnieMConfig)

# Register the dummy module so that AutoConfig / AutoModel can find it
sys.modules["transformers.models.deprecated.ernie_m"] = dummy_module




Patched dummy ErnieMConfig


Semantic Alignment Score

In [8]:

from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# Use a sentence-embedding model from Hugging Face
EMB_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

emb_tokenizer = AutoTokenizer.from_pretrained(EMB_MODEL_NAME)
emb_model = AutoModel.from_pretrained(EMB_MODEL_NAME).to(device)

def embed_sentences(sent_list):
    """
    Encode a list of sentences into L2-normalized embeddings using mean pooling.
    """
    if len(sent_list) == 0:
        return None

    inputs = emb_tokenizer(
        sent_list,
        padding=True,
        truncation=True,
        return_tensors="pt",
    ).to(device)

    with torch.no_grad():
        outputs = emb_model(**inputs)
        last_hidden = outputs.last_hidden_state  # [batch, seq, hidden]
        attention_mask = inputs["attention_mask"]  # [batch, seq]

        # Mean pooling
        mask = attention_mask.unsqueeze(-1).expand(last_hidden.size()).float()
        sum_embeddings = (last_hidden * mask).sum(dim=1)
        sum_mask = mask.sum(dim=1)
        embeddings = sum_embeddings / sum_mask

        # L2 normalize
        embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)

    return embeddings  # [batch, hidden]


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [9]:
def semantic_alignment_score(doc: str, summary: str) -> float:
    """
    Compute a semantic alignment score between a source document and a summary.
    For each summary sentence, we take the maximum cosine similarity to any
    source sentence, then average over all summary sentences.

    Higher score means the summary is more semantically aligned with the source.
    """
    # 1. Sentence split
    doc_sents = [s.strip() for s in nltk.sent_tokenize(doc) if s.strip()]
    sum_sents = [s.strip() for s in nltk.sent_tokenize(summary) if s.strip()]

    if not doc_sents or not sum_sents:
        return 0.0

    # 2. Encode sentences
    doc_embs = embed_sentences(doc_sents)    # [N_doc, dim]
    sum_embs = embed_sentences(sum_sents)    # [N_sum, dim]

    if doc_embs is None or sum_embs is None:
        return 0.0

    # 3. Cosine similarity matrix: [N_sum, N_doc]
    # Since embeddings are L2-normalized, dot product = cosine similarity
    cos_sim = torch.matmul(sum_embs, doc_embs.T)  # [N_sum, N_doc]

    # 4. For each summary sentence, take max similarity to any source sentence
    max_per_sum_sent, _ = torch.max(cos_sim, dim=1)  # [N_sum]

    # 5. Average over summary sentences
    score = float(max_per_sum_sent.mean().cpu().item())
    return score


In [10]:
semantic_scores = [
    semantic_alignment_score(source_document, s)
    for s in candidate_summaries
]

print("=== Semantic alignment scores (higher is better) ===")
for i, (s, sc) in enumerate(zip(candidate_summaries, semantic_scores)):
    print(f"[Cand {i}] score = {sc:.4f}")
    print(s)
    print("-" * 80)


=== Semantic alignment scores (higher is better) ===
[Cand 0] score = 0.9865
– Joe Biden met with congressional leaders at the White House today to discuss a new economic relief package, reports the AP. The meeting comes as inflation remains high and lawmakers are divided over the size of government spending. Meanwhile, the Federal Reserve signaled it may slow down the pace of interest rate hikes later this year if price increases continue to cool.
--------------------------------------------------------------------------------
[Cand 1] score = 0.9886
– Joe Biden met with congressional leaders at the White House today to discuss a new economic relief package, reports the Hill. The meeting comes as inflation remains high and lawmakers are divided over the size of government spending. Meanwhile, the Federal Reserve signaled it may slow down the pace of interest rate hikes later this year if price increases continue to cool.
----------------------------------------------------------------